In [1]:
from sentence_transformers import SentenceTransformer, util
import openai

In [2]:
# 1. Ler o conteúdo do arquivo JSON com as jurisprudências
import json

with open("data_ementa_will_4meses.json", "r", encoding="utf-8") as f:
    juris_list = json.load(f)

# Verificar quantas jurisprudências foram carregadas
print(f"Total de jurisprudências carregadas: {len(juris_list)}")

Total de jurisprudências carregadas: 5658


In [3]:
# 2. Carregar o modelo de embeddings da Sentence Transformers
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [4]:
# 3. Codificar todas as jurisprudências com o modelo
import torch

juris_embeddings = []

for i, texto in enumerate(juris_list):
    print(f"Processando jurisprudência {i+1}/{len(juris_list)}")
    emb = model.encode(texto, convert_to_tensor=True)
    juris_embeddings.append(emb)

# Converter para tensor único
juris_embeddings = torch.stack(juris_embeddings)

Processando jurisprudência 1/5658
Processando jurisprudência 2/5658
Processando jurisprudência 3/5658
Processando jurisprudência 4/5658
Processando jurisprudência 5/5658
Processando jurisprudência 6/5658
Processando jurisprudência 7/5658
Processando jurisprudência 8/5658
Processando jurisprudência 9/5658
Processando jurisprudência 10/5658
Processando jurisprudência 11/5658
Processando jurisprudência 12/5658
Processando jurisprudência 13/5658
Processando jurisprudência 14/5658
Processando jurisprudência 15/5658
Processando jurisprudência 16/5658
Processando jurisprudência 17/5658
Processando jurisprudência 18/5658
Processando jurisprudência 19/5658
Processando jurisprudência 20/5658
Processando jurisprudência 21/5658
Processando jurisprudência 22/5658
Processando jurisprudência 23/5658
Processando jurisprudência 24/5658
Processando jurisprudência 25/5658
Processando jurisprudência 26/5658
Processando jurisprudência 27/5658
Processando jurisprudência 28/5658
Processando jurisprudência 29

In [5]:
# 4. Pergunta do usuário
pergunta = "O TJGO possui jurisprudência sobre responsabilidade civil do Estado por falha no atendimento hospitalar, gerando dano moral?"

In [6]:
# 5. Gerar o embedding da pergunta
pergunta_embedding = model.encode(pergunta, convert_to_tensor=True)

In [7]:
# 6. Calcular similaridade e recuperar jurisprudência mais próxima
from sentence_transformers import util

similaridades = util.cos_sim(pergunta_embedding, juris_embeddings)[0]
indice_mais_relevante = similaridades.argmax().item()
juris_relevante = juris_list[indice_mais_relevante]

print(f"Jurisprudência mais relevante:\n\n{juris_relevante}")

Jurisprudência mais relevante:

Número do processo: 5601639-41.2019.8.09.0093
Câmara Cível
1ª Câmara Cível - MURILO VIEIRA DE FARIA - (DESEMBARGADOR)
07/02/2025 08:52:12
EMENTA: DIREITO ADMINISTRATIVO E PROCESSUAL CIVIL. APELAÇÕES CÍVEIS. RESPONSABILIDADE CIVIL DO ESTADO. FALHA NA PRESTAÇÃO DE SERVIÇOS MÉDICOS. RESSARCIMENTO DE DESPESAS MÉDICO-HOSPITALARES. DANO MORAL NÃO CONFIGURADO. PARCIAL PROVIMENTO. I. CASO EM EXAME:1. Apelações Cíveis interpostas pelo Estado de Goiás e pelo autor contra sentença que condenou o Estado ao pagamento de danos materiais e morais em ação de responsabilidade civil. O autor buscava majoração dos danos morais, enquanto o Estado questionava a responsabilização e os valores fixados. II. QUESTÃO EM DISCUSSÃO2. Há duas questões em discussão: (I) verificar se o Estado de Goiás é responsável pelo ressarcimento das despesas médico-hospitalares particulares realizadas pelo autor, diante da inexistência de serviço de atendimento público de urgência; (II) analisar 

In [8]:
# 7. Criar prompt para o modelo
prompt = f"""
Abaixo está uma decisão judicial completa. Responda à pergunta do usuário e, em seguida, reproduza o conteúdo do acórdão sem omissões.

[DOCUMENTO JURÍDICO RELEVANTE]
{juris_relevante}

[PERGUNTA DO USUÁRIO]
{pergunta}

[RESPOSTA:]
"""

In [9]:
# 8. Chamada à API do modelo sabia-3
import openai

client = openai.OpenAI(
    api_key="COLOCAR_CHAVE_API",
    base_url="https://chat.maritaca.ai/api",
)

response = client.chat.completions.create(
    model="sabia-3",
    messages=[
        {
            "role": "system",
            "content": "Você é um assistente jurídico. O conteúdo abaixo é uma jurisprudência completa fornecida pelo usuário. Use esse conteúdo na íntegra para responder à pergunta, e transcreva integralmente o conteúdo do acórdão apresentado, se solicitado."
        },
        {
            "role": "user",
            "content": prompt
        },
    ],
    max_tokens=8000
)

In [10]:
# 9. Mostrar resposta da API
response_text = response.choices[0].message.content
print(response_text)

O Tribunal de Justiça do Estado de Goiás (TJGO) possui jurisprudência sobre responsabilidade civil do Estado por falha no atendimento hospitalar, mas no caso específico analisado no acórdão fornecido, o dano moral não foi configurado. A decisão judicial aborda a responsabilidade civil do Estado de Goiás por omissão, fundamentada no art. 37, §6o, da Constituição Federal, e reconhece a necessidade de ressarcimento das despesas médico-hospitalares particulares devido à ausência de atendimento público de urgência adequado. No entanto, o sofrimento alegado pelo autor foi considerado inerente à gravidade da condição médica enfrentada e não como uma violação específica à sua personalidade ou dignidade, o que é necessário para configurar o dano moral.

Aqui está o conteúdo integral do acórdão, conforme solicitado:

Número do processo: 5601639-41.2019.8.09.0093
Câmara Cível
1a Câmara Cível - MURILO VIEIRA DE FARIA - (DESEMBARGADOR)
Data: 07/02/2025 08:52:12

EMENTA: DIREITO ADMINISTRATIVO E PRO